In [1]:
import pandas as pd
import torch
from model.FieldingModel import FieldingModel


/home/wahoo/.pyenv/versions/3.13.2/envs/ootp-analytics/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
league =  'BZ'
season_start = 2016
season_end = 2017
positions = [9,8,7,6,5,4,3,2]
ratings_type = "Standard"

In [3]:

    
for position in positions:
    print(f"\r\nTraining model for position {position}")
    fieldingModel = FieldingModel(league, season_start, season_end, position, ratings_type)
    fieldingModel.load_data(position)

    epoch_count, loss = fieldingModel.train(5000)
    print(f"Epoch [{epoch_count}/{5000}], Loss: {loss:.7f}")

    test_loss = fieldingModel.evaluate()
    print(f"Test Mean Squared Error: {test_loss:.7f}")

    feature_importance = fieldingModel.feature_importance()
    print(feature_importance)

    fieldingModel.save_model()


Training model for position 9
CUDA is available! Using GPU.
(53, 9)
(107, 9)
Empty DataFrame
Columns: [Age, WT, SPE, OFRngDelta, OFArmDelta, OFERRDelta, RF, ID, runsPAdjSeason]
Index: []
Epoch [2000/5000], Best Loss: 0.2012588, Best Eval: 10000.0000000
Epoch [4000/5000], Best Loss: 0.1508984, Best Eval: 10000.0000000
Epoch [4999/5000], Loss: 0.2273796
Test Mean Squared Error: 14.4104652
   mean_abs_shap  stdev_abs_shap        name
3       8.825935        5.562378  OFRngDelta
4       3.489429        4.231469  OFArmDelta
0       3.471535        3.327341         Age
1       2.936488        3.761741          WT
2       2.675181        2.424312         SPE
5       2.431771        2.055375  OFERRDelta
7       2.422380        2.439482          ID
6       1.683224        1.980732          RF

Training model for position 8
CUDA is available! Using GPU.
(52, 8)
(106, 8)
Empty DataFrame
Columns: [Age, WT, SPE, OFRngDelta, OFArmDelta, OFERRDelta, CF, runsPAdjSeason]
Index: []
Epoch [2000/5000], B